# ACP Project - Model Evaluation

## Setup

In [1]:
import warnings, pickle, os
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

from IPython.display import display
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(10,10)})

import shap
import optuna

%load_ext autoreload
%autoreload 1

In [2]:
from dataset import SCIData, SCICols
%aimport utils.evaluation, dataset

In [71]:
from sklearn.model_selection import train_test_split

sci = SCIData.load('data/sci.h5')

scii = (
    SCIData(SCIData.quickload("data/sci_processed.h5").sort_values("AdmissionDateTime"))
    .mandate(SCICols.news_data_raw)
    .derive_critical_event(within=1, return_subcols=True)
    .augment_shmi(onehot=True)
  #  .omit_redundant()
    .derive_ae_diagnosis_stems(onehot=False)
   # .categorize()
   # .onehot_encode_categories()
)

sci_train, sci_test, _, y_test_mortality, _, y_test_criticalcare = train_test_split(
    scii,
    scii.DiedWithinThreshold,
    scii.CriticalCare,
    test_size=0.33,
    random_state=42,
    shuffle=False,
)
sci_train, sci_test = SCIData(sci_train), SCIData(sci_test)

## Population

In [73]:
pd.DataFrame(
    [
        dict(
            Subset=name,
            Start=df.AdmissionDateTime.min().strftime('%Y-%m-%d'),
            End=df.AdmissionDateTime.max().strftime('%Y-%m-%d'),
            Records=df.shape[0],
            Patients=df.PatientNumber.unique().shape[0],
            CriticalEvents=f'{df.CriticalEvent.sum()} ({round(df.CriticalEvent.sum()/df.shape[0]*100,2)}%)',
            Deaths=f'{df.DiedWithinThreshold.sum()} ({round(df.DiedWithinThreshold.sum()/df.shape[0]*100,2)}%)',
            CriticalCare=f'{df.CriticalCare.sum()} ({round(df.CriticalCare.sum()/df.shape[0]*100,2)}%)',
            Readmissions=f'{df.Readmission.sum()} ({round(df.Readmission.sum()/df.shape[0]*100,2)}%)',
        )
        for name, df in {'Training Set': sci_train, 'Testing Set': sci_test, 'Total': scii}.items()
    ]
).set_index('Subset').T

Subset,Training Set,Testing Set,Total
Start,2015-01-01,2019-08-10,2015-01-01
End,2019-08-10,2022-03-31,2022-03-31
Records,81285,40036,121321
Patients,44789,25579,62211
CriticalEvents,2182 (2.68%),932 (2.33%),3114 (2.57%)
Deaths,224 (0.28%),118 (0.29%),342 (0.28%)
CriticalCare,1986 (2.44%),828 (2.07%),2814 (2.32%)
Readmissions,10764 (13.24%),4674 (11.67%),15438 (12.72%)


1. Data size (total, training, testing, total patients)
2. Outcomes (critical, deaths, icu)
3. Sex (female %), Age (median)
4. Comorbid conditions (median #, max 7 per patient)
5. Presence of vital signs, presence of blood test results, 
6. LOS of hospitalisation 

In [74]:
pd.DataFrame(
    [
        {
            k:f'{round(df[v].isna().sum().sum()/df[v].size*100,2)}%' for k, v in df.feature_groups.items()
        } | dict(Subset=name) for name, df in {'Training Set': sci_train, 'Testing Set': sci_test, 'Total': scii}.items()
    ]
).set_index('Subset').T

Subset,Training Set,Testing Set,Total
news,0.0%,0.0%,0.0%
news_scores,0.0%,0.0%,0.0%
news_extended,4.95%,2.9%,4.27%
phenotype,0.0%,0.0%,0.0%
ae,47.03%,56.15%,50.04%
labs,61.71%,19.65%,47.83%
hospital,0.0%,0.0%,0.0%
diagnoses,2.47%,2.14%,2.36%
